## 딥러닝 실습 과제 2주차 - CNN을 활용하여 보드 전체를 입력받아 9칸 상태 예측

다음  세 가지 활동을 해봅시다.

01. **모델 설계**: CNN 기반 분류 모델 활용
02. **손실 함수 정의**: 각 칸에 대해 3-클래스 분류
03. **학습 및 평가**: 시각화 함수, 정확도 측정, confusion matrix 등

TTTDataset.zip을 불러와 문제에서 요하는 코드를 구현하세요.

💡 **데이터 구조**  
- **`image_black`** : 이미지 데이터  
- **`labels`** : 타겟 데이터  

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import json

## 00. 클래스
정의한 클래스를 이용해 실행해 주세요.

In [ ]:
class TTTDataset(Dataset):
    def __init__(self, image_paths, label_paths, transform=None):
        """
        틱택토 데이터셋을 PyTorch Dataset 형태로 변환.
        :param image_paths: 이미지 파일 경로 리스트
        :param label_paths: 레이블 JSON 파일 경로 리스트
        :param transform: 이미지 전처리 변환
        """
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.transform = transform
        self.data = self._load_data()


    def _load_data(self):
        """ 이미지 & 레이블 로드 """
        data = []
        for img_path, lbl_path in zip(self.image_paths, self.label_paths):
            # 이미지를 흑백(Grayscale)로 변환
            image = Image.open(img_path).convert("L")  # "RGB" 대신 "L" 사용

            # JSON 레이블 로드
            with open(lbl_path, 'r') as f:
                labels = json.load(f)

            # 레이블을 숫자로 변환 (O=1, X=-1, blank=0)
            label_tensor = torch.tensor(
                [1 if v == "O" else -1 if v == "X" else 0 for v in labels.values()],
                dtype=torch.float32
            )
            data.append((image, label_tensor))

        return data


    def __len__(self):
        """ 데이터셋 크기 반환 """
        return len(self.data)


    def __getitem__(self, idx):
        """ 데이터셋에서 idx 번째 샘플(이미지 & 레이블)을 가져오는 역할 """
        image, label = self.data[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

## 01. 모델 설계: CNN 기반 분류 모델 활용
- 입력: 틱택토 보드 전체 이미지 (예: 128x128)
- 출력: 9개의 상태(각 칸마다 O, X, blank 중 하나)

## 02. 손실 함수 정의: 각 칸에 대해 3-클래스 분류
- 다중 클래스 분류이므로 CrossEntropyLoss 사용 가능
- 9개 칸을 각각 분류하는 방식으로 모델 구성

## 03. 학습 및 평가: 시각화 함수, 정확도 측정, confusion matrix 등
- 학습 데이터로 모델을 훈련하고, 정확도 측정
- 예측이 잘 되는지 시각화하여 분석